#EXPLORE - Expeditions - peaks
####Find Data Quality Issues in:
>_himalaya.bronze.expeditions_peaks_

###Checks
0. Data Overview
1. Nulls
2. Duplicates
3. Distincts
4. Formats
5. Timeliness
___

> ##0. Data Overview

In [0]:
import pyspark.sql.functions as F
from datetime import date

In [0]:
df = spark.table("himalaya.bronze.expeditions_peaks")

In [0]:
print(f"Rows: {df.count()}")
print(f"Columns: {len(df.columns)}")
display(df.limit(10))

> ## 1. Nulls
- ####_Large Dataset_
  - The data set is too large to make sense right now. I will figure out something later

In [0]:
for c in df.columns:
  null_count = df.filter(F.col(c).isNull()).count()
  status = "✅" if null_count == 0 else "❌"
  print(f"{status} Nulls in {c}: {null_count}")

In [0]:
for c in df.columns:
  null_count = df.filter(F.col(c).isNull()).count()
  if null_count != 0:
      print(f"{c}:")
      df.filter(F.col(c).isNull()).show()

> ## 2. Duplicates

In [0]:
total = df.count()
distinct = df.distinct().count()
duplicates = total - distinct

print(f"Total rows:    {total}")
print(f"Distinct rows: {distinct}")
print(f"Duplicates:    {duplicates}")

if duplicates == 0:
    print("✅ No duplicates")
else:
    print(f"❌ {duplicates} duplicate found")

> ## 3. Distinct

- ####⚠️ Large Dataset Again
  - Data does not seem to have any issues just large.

In [0]:
bool_cols = ["msuccess", "mclaimed", "mdisputed", "msolo", "mtraverse", 
             "mski", "mparapente", "mspeed", "bconly", "nottobc", 
             "support", "disabled", "hired", "sherpa", "tibetan",
             "mo2used", "mo2none", "mo2climb", "mo2descent", 
             "mo2sleep", "mo2medical", "death"]

cat_cols = ["mseason", "myear", "peakid", "citizen", "sex", 
            "status", "mroute1", "mroute2", "mroute3",
            "deathtype", "deathclass", "msmtterm"]

all_cols = bool_cols + cat_cols

for c in all_cols:
    distinct_count = df.select(c).distinct().count()
    print(f"{c}: {distinct_count} distinct values")

In [0]:
df.select("sex").distinct().show()

> ## 4. Format

- ####✅ Looks fine

In [0]:
df.printSchema()

>## 5. Timeliness

In [0]:
current_year = date.today().year

# Year range check
min_year, max_year = df.select(F.min("myear"), F.max("myear")).first()
print(f"Year     — Min: {min_year} | Max: {max_year}")
print("✅ Max year not in future" if max_year <= current_year else "❌ Max year is in the future")

# YOB range check
min_yob, max_yob = df.select(F.min("yob"), F.max("yob")).first()
print(f"YOB      — Min: {min_yob} | Max: {max_yob}")
print("✅ Max YOB not in future" if max_yob <= current_year else "❌ Max YOB is in the future")

# msmtdate range check
min_smt, max_smt = df.select(F.min("msmtdate1"), F.max("msmtdate1")).first()
print(f"Smt Date — Min: {min_smt} | Max: {max_smt}")
print("✅ Max summit date not in future" if str(max_smt) <= str(date.today()) else "❌ Max summit date is in the future")

# Cross checks — row level
print("\n--- Cross Checks ---")

# yob should be less than myear
invalid_yob = df.filter(F.col("yob") >= F.col("myear")).count()
print("✅ YOB < expedition year" if invalid_yob == 0 else f"❌ {invalid_yob} rows where YOB >= expedition year")

# msmtdate1 year should be >= myear
invalid_smt_year = df.filter(
    F.col("msmtdate1").isNotNull() & 
    (F.year(F.to_date("msmtdate1")) < F.col("myear"))
).count()
print("✅ Summit date >= expedition year" if invalid_smt_year == 0 else f"❌ {invalid_smt_year} rows where summit date < expedition year")

# msmtdate1 year should be > yob
invalid_smt_yob = df.filter(
    F.col("msmtdate1").isNotNull() & 
    (F.year(F.to_date("msmtdate1")) <= F.col("yob"))
).count()
print("✅ Summit date > YOB" if invalid_smt_yob == 0 else f"❌ {invalid_smt_yob} rows where summit date <= birth year")

___
# Issues Found
### 1. Large Dataset
- Will take care of it

### 2. Duplicates
- ✅ No duplicates.

### 3. Distinct
- ⚠️ `sex` — M/F/X
- ⚠️ Will look through others in silver 

### 4. Format
- ⚠️ `date` — stored as string. Will cast to DateType in Silver.

### 5. Timeliness
- ⚠️ Various Cross Check Problems